<a href="https://colab.research.google.com/github/taduriv/NLPCW2/blob/main/DeeplearningCWF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [97]:

data1=pd.DataFrame(pd.read_table(r"/content/attending_event.txt"))
data2=pd.DataFrame(pd.read_table(r"/content/communication_issue.txt"))
data3=pd.DataFrame(pd.read_table(r"/content/going_to_places.txt"))
data4=pd.DataFrame(pd.read_table(r"/content/legal_issue.txt"))
data5=pd.DataFrame(pd.read_table(r"/content/money_issue.txt"))
data6=pd.DataFrame(pd.read_table(r"/content/outdoor_activity.txt",encoding='latin1'))
data7=pd.DataFrame(pd.read_table(r"/content/personal_care.txt"))
data8=pd.DataFrame(pd.read_table(r"/content/physical_pain.txt"))


In [98]:
data2['ID']=' '
first_column = data2.pop('ID')
data2.insert(0, 'ID', first_column)

In [99]:
data1.columns=['ID', 'text', 'polarity', 'Event']
data2.columns=['ID', 'text', 'polarity', 'Event']
data3.columns=['ID', 'text', 'polarity', 'Event']
data4.columns=['ID', 'text', 'polarity', 'Event']
data5.columns=['ID', 'text', 'polarity', 'Event']
data6.columns=['ID', 'text', 'polarity', 'Event']
data7.columns=['ID', 'text', 'polarity', 'Event']
data8.columns=['ID', 'text', 'polarity', 'Event']

In [100]:
Data=pd.concat([data1, data2, data3, data4, data5, data6, data7, data8], ignore_index=True)


In [106]:
Data['polarity'] = Data['polarity'].str.strip()  # Remove leading/trailing whitespace
Data['polarity'] = Data['polarity'].replace({
    'POSIITVE': 'POSITIVE',
    'POSTITIVE': 'POSITIVE'
})


In [107]:

#removing the spaces in the text and replacing the few texts
Data['Event'] = Data['Event'].str.strip()
Data['Event'] = Data['Event'].replace({
    'OOUTDOOR_ACTIVITY': 'OUTDOOR_ACTIVITY',
    'PERSONA_CARE': 'PERSONAL_CARE'
})

In [111]:
Data['polarity'].unique()

array(['NEGATIVE', 'NEUTRAL', 'POSITIVE'], dtype=object)

In [112]:
Data['Event'].unique()

array(['ATTENDING_EVENT', 'COMMUNICATION_ISSUE', 'GOING_TO_PLACES',
       'LEGAL_ISSUE', 'MONEY_ISSUE', 'OUTDOOR_ACTIVITY', 'PERSONAL_CARE',
       nan, '(FEAR_OF)_PHYSICAL_PAIN'], dtype=object)

In [141]:
Data

,ID,text,polarity,Event
0,2,Choosing to undergo open heart surgery when I ...,NEGATIVE,ATTENDING_EVENT
1,3,Either we're going to have a big barbecue or w...,NEUTRAL,ATTENDING_EVENT
2,4,"For health reasons, I need to undergo medical ...",NEGATIVE,ATTENDING_EVENT
3,5,"I am going to be at the concert here, and in o...",NEUTRAL,ATTENDING_EVENT
4,6,"I am really excited about the concert, but I c...",POSITIVE,ATTENDING_EVENT
...,...,...,...,...
1267,156,Today I fell down again on it and it was getti...,NEGATIVE,(FEAR_OF)_PHYSICAL_PAIN
1268,157,"What can I say, Larry? I had an accident.",NEGATIVE,(FEAR_OF)_PHYSICAL_PAIN
1269,158,"When I realized that it was an earthquake, it ...",NEGATIVE,(FEAR_OF)_PHYSICAL_PAIN
1270,159,"When I was assaulted, I was 13 and wearing a s...",NEGATIVE,(FEAR_OF)_PHYSICAL_PAIN


In [131]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Sample dataset
texts = Data['text'].values
polarities = Data['polarity'].values
events = Data['Event'].values

# Preprocessing
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=100)

# Encode labels
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(polarities)  # or events
num_classes = np.max(encoded_labels) + 1


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.2)

# Model CNN
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=128, input_length=100))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Accuracy: {accuracy:.4f}")

Epoch 1/5
26/26 [==============================] - 2s 59ms/step - loss: 1.0286 - accuracy: 0.4883 - val_loss: 0.9892 - val_accuracy: 0.5147
Epoch 2/5
26/26 [==============================] - 1s 47ms/step - loss: 0.9676 - accuracy: 0.5486 - val_loss: 0.9623 - val_accuracy: 0.5588
Epoch 3/5
26/26 [==============================] - 1s 50ms/step - loss: 0.8156 - accuracy: 0.6531 - val_loss: 0.9001 - val_accuracy: 0.6422
Epoch 4/5
26/26 [==============================] - 1s 47ms/step - loss: 0.5440 - accuracy: 0.7626 - val_loss: 0.8725 - val_accuracy: 0.6618
Epoch 5/5
8/8 [==============================] - 0s 19ms/step
Accuracy: 0.6510


In [ ]:

import matplotlib.pyplot as plt

# 1. Training & Validation Loss
plt.figure(figsize=(10, 6))
plt.plot(model.history.history['loss'], label='Training Loss')
plt.plot(model.history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 2. Accuracy
plt.figure(figsize=(10, 6))
plt.plot(model.history.history['accuracy'], label='Training Accuracy')
plt.plot(model.history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# 3. Confusion Matrix (requires additional library)
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


In [144]:
polarities

array(['NEGATIVE', 'NEUTRAL', 'NEGATIVE', ..., 'NEGATIVE', 'NEGATIVE',
       'NEGATIVE'], dtype=object)

In [136]:
from keras.layers import LSTM

# Model
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=128, input_length=100))
model.add(LSTM(128))
model.add(Dense(10, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# Predict and evaluate
y_pred_probs = model.predict(X_test)  # Get predicted probabilities for each class
y_pred = np.argmax(y_pred_probs, axis=1)  # Get class with highest probability
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Epoch 1/5
26/26 [==============================] - 8s 247ms/step - loss: 1.0293 - accuracy: 0.4883 - val_loss: 0.9704 - val_accuracy: 0.5147
Epoch 2/5
26/26 [==============================] - 4s 162ms/step - loss: 0.9000 - accuracy: 0.5387 - val_loss: 0.8940 - val_accuracy: 0.5686
Epoch 3/5
26/26 [==============================] - 5s 175ms/step - loss: 0.7317 - accuracy: 0.6900 - val_loss: 0.9560 - val_accuracy: 0.5637
Epoch 4/5
26/26 [==============================] - 6s 217ms/step - loss: 0.5090 - accuracy: 0.8327 - val_loss: 0.9050 - val_accuracy: 0.5931
Epoch 5/5
8/8 [==============================] - 1s 48ms/step
Accuracy: 0.6471


In [146]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import pandas as pd

texts = Data['text'].values
polarity = Data['polarity'].values
event = Data['Event'].values

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the data
inputs = tokenizer(texts.tolist(), return_tensors="pt", padding=True, truncation=True, max_length=100)

# Encode the labels
le_polarity = LabelEncoder()
le_event = LabelEncoder()
polarity_encoded = le_polarity.fit_transform(polarity)
event_encoded = le_event.fit_transform(event)

# Split the data into training and testing sets
X_train, X_test, y_train_polarity, y_test_polarity = train_test_split(inputs['input_ids'], polarity_encoded, test_size=0.2, random_state=42)
_, _, y_train_event, y_test_event = train_test_split(inputs['input_ids'], event_encoded, test_size=0.2, random_state=42)



In [ ]:
polarity_encoded.unique()

In [147]:
# Create attention masks
attention_masks_train = inputs['attention_mask'][:len(X_train)]
attention_masks_test = inputs['attention_mask'][len(X_train):]

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = {'input_ids': X_train, 'attention_mask': attention_masks_train}
test_encodings = {'input_ids': X_test, 'attention_mask': attention_masks_test}

dataset_polarity_train = CustomDataset(train_encodings, y_train_polarity)
dataset_polarity_test = CustomDataset(test_encodings, y_test_polarity)

dataset_event_train = CustomDataset(train_encodings, y_train_event)
dataset_event_test = CustomDataset(test_encodings, y_test_event)



In [149]:
# Define the BERT model for polarity prediction
model_polarity = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le_polarity.classes_))

# Define training arguments
training_args_polarity = TrainingArguments(
    output_dir='./results_polarity',  # Directory to save model checkpoints and results
    num_train_epochs=3,               # Number of training epochs
    per_device_train_batch_size=8,    # Batch size for training
    per_device_eval_batch_size=8,     # Batch size for evaluation
    warmup_steps=500,                 # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # Weight decay for optimizer
    logging_dir='./logs_polarity',    # Directory for storing logs
    logging_steps=10,                 # Log every 10 steps
    eval_strategy="epoch"             # Evaluation strategy (use eval_strategy instead of evaluation_strategy)
)

# Create the Trainer instance
trainer_polarity = Trainer(
    model=model_polarity,                 # The instantiated  Transformers model to be trained
    args=training_args_polarity,          # Training arguments, defined above
    train_dataset=dataset_polarity_train, # Training dataset
    eval_dataset=dataset_polarity_test    # Evaluation dataset
)

# Train the model
trainer_polarity.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,1.019200,0.917731


Epoch,Training Loss,Validation Loss
1,1.019200,0.917731
2,0.557400,0.578549
3,0.477400,0.656507


TrainOutput(global_step=384, training_loss=0.7869691656281551, metrics={'train_runtime': 3419.7339, 'train_samples_per_second': 0.892, 'train_steps_per_second': 0.112, 'total_flos': 156788874509400.0, 'train_loss': 0.7869691656281551, 'epoch': 3.0})

In [150]:
# Define the BERT model for event prediction
model_event = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le_event.classes_))

training_args_event = TrainingArguments(
    output_dir='./results_event',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs_event',
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer_event = Trainer(
    model=model_event,
    args=training_args_event,
    train_dataset=dataset_event_train,
    eval_dataset=dataset_event_test
)

trainer_event.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.139400,2.098008


Epoch,Training Loss,Validation Loss
1,2.139400,2.098008
2,1.257700,1.189211
3,0.549800,0.639076


TrainOutput(global_step=384, training_loss=1.5332234054803848, metrics={'train_runtime': 3419.0148, 'train_samples_per_second': 0.892, 'train_steps_per_second': 0.112, 'total_flos': 156797320897800.0, 'train_loss': 1.5332234054803848, 'epoch': 3.0})

In [151]:
import numpy as np
from sklearn.metrics import accuracy_score

# Evaluate polarity prediction
predictions_polarity = trainer_polarity.predict(dataset_polarity_test)
predicted_labels_polarity = np.argmax(predictions_polarity.predictions, axis=1)
accuracy_polarity = accuracy_score(y_test_polarity, predicted_labels_polarity)
print(f"Polarity Accuracy: {accuracy_polarity:.4f}")

# Evaluate event prediction
predictions_event = trainer_event.predict(dataset_event_test)
predicted_labels_event = np.argmax(predictions_event.predictions, axis=1)
accuracy_event = accuracy_score(y_test_event, predicted_labels_event)
print(f"Event Accuracy: {accuracy_event:.4f}")


Polarity Accuracy: 0.7647


Event Accuracy: 0.8275


In [154]:
# Sample input text for prediction
sample_text = input()

# Tokenize the sample text
inputs = tokenizer(sample_text, return_tensors="pt", padding=True, truncation=True, max_length=100)

# Move models to the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_polarity.to(device)
model_event.to(device)

# Set the models to evaluation mode
model_polarity.eval()
model_event.eval()

# Ensure the inputs are moved to the appropriate device
inputs = {key: val.to(device) for key, val in inputs.items()}

# Get predictions for polarity
with torch.no_grad():
    outputs_polarity = model_polarity(**inputs)
    predicted_label_polarity = torch.argmax(outputs_polarity.logits, dim=1).cpu().numpy()

# Get predictions for event
with torch.no_grad():
    outputs_event = model_event(**inputs)
    predicted_label_event = torch.argmax(outputs_event.logits, dim=1).cpu().numpy()

# Decode the predicted labels
predicted_polarity = le_polarity.inverse_transform(predicted_label_polarity)[0]
predicted_event = le_event.inverse_transform(predicted_label_event)[0]

# Print the results
print(f"Input Text: {sample_text}")
print(f"Predicted Polarity: {predicted_polarity}")
print(f"Predicted Event: {predicted_event}")

If a guy owes me money and I want my money, that doesn't make me an extortionist" said Provenzano, sitting in the home office of his spacious Bergen County home, a copy of Elia Kazan's On What Makes A Director by his side.
Input Text: If a guy owes me money and I want my money, that doesn't make me an extortionist" said Provenzano, sitting in the home office of his spacious Bergen County home, a copy of Elia Kazan's On What Makes A Director by his side.
Predicted Polarity: NEUTRAL
Predicted Event: MONEY_ISSUE
